In [2]:
2+2

4

In [3]:
import os

In [4]:
!pwd

/d/youtube/machine-learning-llm/research


In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\youtube\\machine-learning-llm'

## Entity (/entity/config_entity) 

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


## Config Manager (/config/config_manager)

In [8]:
from llm.logchain_model.constants import * 

In [9]:
from llm.logchain_model.utils.common import read_yaml, create_directories

In [10]:
class ConfigManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## Component Build (/components/data_ingestion.py)

In [11]:
import os
import zipfile
import gdown
from llm.logchain_model import logger
from llm.logchain_model.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        """ Download the data from google drive """
        try: 
            dataset_url = self.config.source_URL
            download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Start downloading data from {dataset_url} into file {download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,download_dir)

            logger.info(f"End download data from {dataset_url} into file {download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


            

## data ingestion (pipeline\01_data_ingestion)

In [13]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-22 09:52:22,018: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-22 09:52:22,030: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-22 09:52:22,034: INFO: common: created directory at: artifacts]
[2023-12-22 09:52:22,034: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-22 09:52:22,034: INFO: 2192327126: Start downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=13b64290-dcbd-4bfd-aff6-93cb714b0fbd
To: d:\youtube\machine-learning-llm\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:02<00:00, 26.7MB/s]


[2023-12-22 09:52:26,070: INFO: 2192327126: End download data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
